In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import sys,os


In [2]:
sim = 'test0'
path = '/Users/ajc/FUSE/simulations/tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/transport-logical/%s/'%sim
Time_ATS = np.loadtxt(os.path.join(path,'surface-outlet.dat'),usecols=0,skiprows=6)
Time_ATS = [t/3600 for t in Time_ATS]
Data_ATS = np.loadtxt(os.path.join(path,'surface-outlet.dat'),usecols=1,skiprows=6)

OSError: /Users/ajc/FUSE/simulations/tests/ats-transport-subgrid-testsuite/tests/logical-surface-twoway/transport-logical/test0/surface-outlet.dat not found.

In [ ]:
A = 0.5496 # cross-sectional area
Q = 0.1 # m^3/second
a, b = 0, 1000
dx = 100
N = int((b-a)/dx) + 1#ncells
print (N)
T = 2*86400 # hour
dt = 450
Nt = int(T/dt) +1
D = 0 #dispersion
X = np.linspace(0,N,N)
print (N)
Vel = Q/A
CFL = (Q/A) * dt /dx 
print ('CFL: ', CFL,Vel)

Time_hr = [i*dt/3600 for i in range(Nt)]

In [ ]:
#subgrid
alpha = 1.7e-5


In [ ]:
#Coef1 = 1 + 2 *dt*D/(dx**2) + dt * Q / (A*dx)
#Coef2 = Q*dt/(A*dx) + dt * D /(dx**2)
#Coef3 = dt * D /(dx**2) 
Coef1 = 1 - (Q * dt) / (A*dx)
Coef2 = (Q * dt) / (A*dx)
r = (dt/dx) *(Q/A)




In [ ]:
Tstep = int(T/dt)

Tau_0 = [3649.63, 1805.57, 1626.54, 1613.26, 1678.51, 1801.08, 1978.1, 2215.79, 2527.87, 2937.04, 3479.01, 4210.42, 5224.11, 6679.84, 8870.96, 12384.6, 18542.1, 30867.6, 61695.7, 185036.]

Tau = np.cumsum(Tau_0)

N_sg = len(Tau) + 1
C_subgrid = np.zeros((Tstep+1,N+1,N_sg))

def subgrid(j_sg,C_s):
    global  C_subgrid
    Gamma = []
    for i in range(N):
        
        for k in range(N_sg):
            if k == 0:
                C_subgrid[j_sg,i,k] = C_s[i]
            elif (k== N_sg -1):
                C_subgrid[j_sg,i,k] = C_subgrid[j_sg-1,i,k-1]
            else:
                #print (i)
                del_tau = Tau[k] - Tau[k-1]
                r_sg = dt/del_tau
                
                C_subgrid[j_sg,i,k] = (1-r_sg) * C_subgrid[j_sg-1,i,k] + r_sg * C_subgrid[j_sg-1,i,k-1]
       
        s1 = np.sum(C_subgrid[j_sg,i,:])/len(Tau)
        
        Gamma.append(s1)
        
    return Gamma
    

In [ ]:

C = np.zeros((Tstep+1,N))
G = np.zeros(N)
for j in range (Tstep):
    #print ('----------------',j)
    for i in range(N):
        
        if i == 0:
            if j*dt <= 86400:
                C[j+1,i] = 1 #- alpha*A*C[j,i]*dt + alpha*G[i]*dt
                #print (alpha*A*C[j,i], alpha*G[i])
            else:
                C[j+1,i] = 0.0 - 0*alpha*C[j,i]*dt + 0*alpha*G[i]*dt
        elif (i==N-1):
            C[j+1,i] = C[j+1,i-1] - 1*alpha*C[j,i]*dt + 1*alpha*G[i]*dt
        else:
            C[j+1,i] = (1-r) * C[j,i] + r * C[j,i-1] - alpha*C[j,i]*dt  + alpha*G[i]*dt #have divided everything by A (cross-sectional area)
            
            
    
    G = subgrid(j+1,C_s=C[j+1,:])
    

In [ ]:
#plt.plot(X,C[Tstep,-1])
%matplotlib qt
plt.plot(Time_ATS,Data_ATS,'r',label='ATS')
plt.plot(Time_hr,C[:,9],'k',label='mine')
plt.legend()

print (np.max(C[:,-1]))